In [9]:
import json
import random

open_file_path = '../../data/edited_objects.json'
out_file_path =''
file_open = open(open_file_path,'r')
file = json.load(file_open)

tar_n = 990 #リストの何番目の要素をtargetとしているか

#大枠はリストだから最初に要素数を指定。全部やるときはfor文にする
target_id = file[tar_n]["id"]
scene_id = file[tar_n]["scene_id"]
#print(target_id)

#それぞれのattributeは階層が１つ深い
attributes = file[tar_n]["attributes"]

#targetはランダムで選ばれたattributesだけをRef_Expリストに格納していく
#その他の同一シーン内のインスタンスは全attributesをリストに格納する
#targetのattributesリストを包含するインスタンスのidをdistractorリストに格納していく
#この一連の流れはできればモジュール化したい（データセットのversionをあげても使いたい）
target_label = file[tar_n]['label']
Ref_Exp = [target_label]
distractor_list = []

#要素番号もリストで持つようにする
distractor_index_list = []

#ランダムで含ませるAttributesに関するFRAG
#lexicalとstate,other,style,symmetoryはデータセットv1では無視する
color_flag = random.randrange(2)
shape_flag = random.randrange(2)
size_flag = random.randrange(2)
material_flag = random.randrange(2)
texture_flag = random.randrange(2)

#ここはもっと綺麗に描きたい（extendの時に条件分岐したい）
color = []
shape = []
size = []
material = []
texture = []

if color_flag == 1:
    #colorなしの場合は空リスト
    color = attributes.get('color',[])

if shape_flag == 1:
    shape = attributes.get('shape',[])

if size_flag == 1:
    size = attributes.get('size',[])

if material_flag == 1:
    material = attributes.get('material',[])

if texture_flag == 1:
    texture = attributes.get('texture',[])

Ref_Exp.extend(color)
Ref_Exp.extend(shape)
Ref_Exp.extend(size)
Ref_Exp.extend(material)
Ref_Exp.extend(texture)

print("シーンID:", scene_id)
print("ターゲットID:", target_id)
print("Referring Expression:", Ref_Exp)

#のちに包含か調べるためにset型に変換
Ref_Exp_Set = set(Ref_Exp)
start = False
for object in file:
    if object["scene_id"] == scene_id:
        start = True
    #scene_idが同じやつは必ず連続なのでforの途中で違くなったらbreakして大丈夫
    if start and object["scene_id"] != scene_id:
        break
    
    object_id = object["id"]
    object_label = object["label"]
    Obj_Exp = [object_label]
    obj_attributes = object["attributes"]

    obj_color = obj_attributes.get('color',[])
    obj_shape = obj_attributes.get('shape',[])
    obj_size = obj_attributes.get('size',[])
    obj_material = obj_attributes.get('material',[])
    obj_texture = obj_attributes.get('texture',[])
 
    Obj_Exp.extend(obj_color)
    Obj_Exp.extend(obj_shape)
    Obj_Exp.extend(obj_size)
    Obj_Exp.extend(obj_material)
    Obj_Exp.extend(obj_texture)

    Obj_Exp_Set = set(Obj_Exp)

    #print(Ref_Exp_Set <= Obj_Exp_Set)
    if Ref_Exp_Set <= Obj_Exp_Set:
        #print(object_id)
        distractor_list.append(object_id)
        distractor_index_list.append(file.index(object))
        
print("条件を満たすインスタンスのIDリスト:",distractor_list)
print("条件を満たすインスタンスの要素番号リスト:",distractor_index_list)
uncertainty = len(distractor_list) - 1
print("不確定性:", uncertainty)

#distractorリストの大きさによってREを削除
#最後に同一シーンでREの重複があれば削除する

シーンID: ab835fae-54c6-29a1-995e-b06cfc555786
ターゲットID: 18
Referring Expression: ['door', 'black', 'wooden']
条件を満たすインスタンスのIDリスト: ['18']
条件を満たすインスタンスの要素番号リスト: [990]
不確定性: 0


In [ ]:
#どのattributeを対象として分割すると不確定性の期待値が最小となるかを計算
#対象はFLAGが0のattribute
#set型で全ての要素を重複なしで取得
#
